# Import Library

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise import accuracy

# Import Data User, Anime, UserAnime

In [3]:
folder = "../Dataset SKRIPSI/"

filename = folder + "users_data"
UsersDF = pickle.load(open(filename,'rb'))

filename = folder + "animes_data"
AnimesDF = pickle.load(open(filename,'rb'))

filename = folder + "scores_data"
ScoresDF = pickle.load(open(filename,'rb'))

# Mengkalikan Matriks User-Item dengan Status

Jika Rating[i] dan Status[i] tidak ada
maka Rating[i] * 2

In [4]:
ScoresDF

,username,anime_id,my_score,my_status,animes_rated
0,Tsundora,21,2,4,259
1,Tsundora,59,5,2,259
2,Tsundora,210,5,2,259
3,Tsundora,249,7,4,259
4,Tsundora,269,6,2,259
...,...,...,...,...,...
193267,Senpai_Helix,4214,10,2,4
193268,Senpai_Helix,18677,9,2,4
193269,Kiyoshi-Kun,1722,10,2,3
193270,Kiyoshi-Kun,2993,10,2,3


In [5]:
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 2, 5, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 3, 2, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 6, 2, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 1, 3, ScoresDF['my_status'])
ScoresDF['my_status'] = np.where(ScoresDF['my_status'] == 4, 1, ScoresDF['my_status'])

In [6]:
ScoresDF.loc[ScoresDF['my_status'] == None]

,username,anime_id,my_score,my_status,animes_rated


In [7]:
ScoresDF['scoreXstatus'] = ScoresDF['my_score'] * ScoresDF['my_status']
ScoresDF['scoreXstatus'] = ScoresDF['scoreXstatus'] / 5
ScoresDF

,username,anime_id,my_score,my_status,animes_rated,scoreXstatus
0,Tsundora,21,2,1,259,0.4
1,Tsundora,59,5,5,259,5.0
2,Tsundora,210,5,5,259,5.0
3,Tsundora,249,7,1,259,1.4
4,Tsundora,269,6,5,259,6.0
...,...,...,...,...,...,...
193267,Senpai_Helix,4214,10,5,4,10.0
193268,Senpai_Helix,18677,9,5,4,9.0
193269,Kiyoshi-Kun,1722,10,5,3,10.0
193270,Kiyoshi-Kun,2993,10,5,3,10.0


# Membuat User-Item Matriks Berdasarkan Rating
Serta mengisi data yang kosong dengan 0

In [8]:
UserItemRatingMatrix = ScoresDF.pivot(index='username',columns='anime_id',values='scoreXstatus')
UserItemRatingMatrix = UserItemRatingMatrix.replace(np.nan,0)
UserItemRatingMatrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,37277,37283,37320,37324,37341,37408,37433,37505,37672,37719
username,,,,,,,,,,,,,,,,,,,,,
-Axel-,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Belatriz-,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-INFERNO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Jhon-,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-Sato-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zerorider,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziomalos,9.0,8.0,9.0,7.0,0.0,8.0,9.0,8.0,8.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zipyto,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training dan Test Model
Training dan test model dipisah 80% : 20%
# SVD

In [9]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(ScoresDF[['username','anime_id','scoreXstatus']],reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=50)

In [10]:
algo_svd = SVD()
prediction_mf = algo_svd.fit(trainset).test(testset)

prediction_mf

[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.456670405786267, details={'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=8.734166042344935, details={'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=9.863885844702338, details={'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=7.956550297428011, details={'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=8.127662986609657, details={'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=4.2, est=4.442371447061568, details={'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=9.662299260953969, details={'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=7.618993499770312, details={'was_impossible': False}),
 Prediction(uid='Vanroy17', iid=58, r_ui=8.0, est=7.102296216812936, details={'was_impossible': False}),
 Prediction(uid='Ammes', iid=814, r_ui=9.0,

In [11]:
recom_svd = algo_svd.predict(uid='Scieska', iid=58)
recom_svd

Prediction(uid='Scieska', iid=58, r_ui=None, est=6.670050959456935, details={'was_impossible': False})

# KNN

In [12]:
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }
algo_knn = KNNBasic(k=50, sim_options=sim_options)
prediction_knn = algo_knn.fit(trainset).test(testset)

# Prediksi
prediction_knn

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


[Prediction(uid='Shouichirou', iid=12679, r_ui=8.0, est=7.599974212579081, details={'actual_k': 21, 'was_impossible': False}),
 Prediction(uid='aleigh', iid=2904, r_ui=9.0, est=9.036749010103486, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='bigkfc2k8', iid=267, r_ui=6.0, est=8.122594412177934, details={'actual_k': 33, 'was_impossible': False}),
 Prediction(uid='Torah', iid=6956, r_ui=10.0, est=7.4385825907359395, details={'actual_k': 50, 'was_impossible': False}),
 Prediction(uid='godT', iid=63, r_ui=7.0, est=7.644712396424897, details={'actual_k': 44, 'was_impossible': False}),
 Prediction(uid='lightofzeref', iid=36511, r_ui=4.2, est=4.138707315164093, details={'actual_k': 10, 'was_impossible': False}),
 Prediction(uid='ranmaSA', iid=401, r_ui=10.0, est=8.263178856129334, details={'actual_k': 29, 'was_impossible': False}),
 Prediction(uid='Josay', iid=16664, r_ui=9.0, est=8.539622888249882, details={'actual_k': 24, 'was_impossible': False}),
 Prediction(uid='Va

In [13]:
recom_knn = algo_knn.predict(uid='Scieska', iid=58)
recom_knn

Prediction(uid='Scieska', iid=58, r_ui=None, est=7.331258408026359, details={'actual_k': 15, 'was_impossible': False})

# Testing Akurasi

In [14]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), KNNBasic()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,1.813486,8.972613,0.682437
KNNBasic,1.900147,0.938813,10.331386
